In [246]:
%pwd

'/Users/lebronaldea1/Downloads'

In [308]:
# Module Imports
import pandas as pd
import re
import sys
import numpy as npd
import collections

In [247]:
#READ ALL OF THESE FILES ONCE 
wd_tax="/usr/mic/post1/metagenomics/ref_sets/fasta/01012015update/taxonomy/2017tax/"
#Human Genes accession
#to check if gene is human
#loc of file: wd_tax + "human_gene_accession"
human_genes=pd.read_table("human_gene_accession",header=None)

#accession to taxid
#loc of file: wd_tax + "nucl_gb.accession2taxid"
acc_to_tax=pd.read_table('nucl_gb.accession2taxid')

#taxinfo
#to check if taxid is from an eukaryota
#loc of file: wd_tax + 'categories.dmp'
taxid_to_k=pd.read_table("categories.dmp", header=None, names=["Domain","sptaxid","taxid"])

In [369]:
#Get taxonomy information
taxnames=pd.read_table('names.dmp.txt', header=None, names=["taxid","","name","","variant","","name_class",""])

In [355]:
taxnames=taxnames[["taxid","name","name_class"]].loc[taxnames["name_class"]=="scientific name"]

In [248]:

#find a way to get the original file !!!!!IMPORTANT

#to get input name : sys.argv
file="874244.fa.frag.cleaned.db.humanid.9606.fna.fasta.txt"
seqid=file.split(".")[0]
threshold= 1e-10

#Get sequence information from catalog
catalog=pd.read_table("catalog.prot.txt", header=None, names=["name","seqid","taxid"])
catalog["seqid"]=catalog["seqid"].astype(str)
seq_taxid=catalog["taxid"][catalog["seqid"]==seqid][0]

In [249]:
#Retrieve information from file
info=[]
frag=0 #Number of fragments
with open(file) as fp:
    for line in fp:
        match=re.search(r'^seq.(\d+)',line)
        if match:
            frag+=1
            seq_frag=match.group(1)
            components=line.split('\t')
            match2=re.search('gi\|\d+\|(.*?)\|(.*?)\|',components[1])
            accgene=match2.group(2)
            evalue=components[10]
            info.append([int(seq_frag),accgene,float(evalue)])

df=pd.DataFrame(info,columns=['frag','accession','evalue'])
#Keep significant e-values and examine
df=df[df['evalue'] <= threshold]

#check if any true then there are some human genes 
check=df['accession'].isin(human_genes[0])

if(any(check)==True):
    delete=list(set(df["frag"][check]))
else:
    delete=[]


## Check if Eukaryota
### use acc_to_tax to get taxid and taxid_to_k to check if eukaryota

In [251]:
#Get  the taxid
set_acc=set(df["accession"])
indices=[i for i, item in enumerate(acc_to_tax['accession.version']) if item in set_acc]
gene_taxid=list(acc_to_tax['taxid'][indices].astype(int))

# Get domain for each taxid
index_dict = dict((value, idx) for idx,value in enumerate(taxid_to_k['taxid']))
indices_dom=[index_dict[x] for x in gene_taxid]
gene_domain=taxid_to_k['Domain'][indices_dom]  
gene_info=pd.DataFrame(np.column_stack([list(acc_to_tax['accession.version'][indices]),gene_taxid,gene_domain]), columns=['gene.acc','taxid','Domain'])



In [291]:
#If the seq_taxid is one of the matches then we can erase from the list, we are trying to report the false positives
check=seq_taxid in gene_info["taxid"]
if check:
    gene_info=gene_info.loc[gene_info["taxid"] != seq_taxid]

#Remove the Eukaryota Fragments from the original file
euk_frag=list(set(df[df["accession"].isin(gene_info["gene.acc"].tolist())]["frag"].tolist()))

delete=delete+euk_frag #add the human fragments
##gene_info.loc[gene_info["Domain"]=="E"]["gene.acc"].tolist()

In [367]:
#how many fragments were labeled as that gene
pairs=set(list(zip(df["frag"], df["accession"])))
counts=dict()
for pair in pairs:
    #pair[1]=gene_name
    if pair[1] not in counts:
        counts[pair[1]]=1
    else:
        counts[pair[1]]+=1

gene_info["counts"]=[counts[gene] for gene in gene_info["gene.acc"]]

# get taxname for taxid of genes.
gene_info["tax_name"]=[taxnames["name"][taxnames["taxid"]==taxid].tolist()[0] for taxid in gene_info["taxid"]]
#Print report
#Report file - ?Report removed sequences?
gene_info.to_csv(seqid+"_blast.report.txt", index=None, sep=' ', mode='a')

In [370]:
gene_info.head()

,gene.acc,taxid,Domain,counts,tax_name
0,X55786.1,3555,E,1,Beta vulgaris subsp. vulgaris
1,M57478.1,3888,E,1,Pisum sativum
2,D37958.1,3750,E,1,Malus domestica
3,Y12731.1,3906,E,1,Vicia faba
4,AJ130748.1,3262,E,1,Equisetum hyemale


## Create fasta v2

In [303]:
#Re-write the frag file without the eukaryota and human fragments
f=open(seqid+'.fa.frag.txt',"r")
p=open(seqid+'_v2.fa.frag',"w")
#Add line specifying how many human genes were found:
notice="Human fragments="+len(delete)+",Proportion:"+round(len(delete)/float(frag),4)+"\n"
p.write(notice)

for line in f:
    match=re.search('^>seq.(\d+)',line)
    if match and int(match.group(1)) in delete: 
        next(f)
        #next #skips sequence line
    else:
        p.write(line)
f.close()